Concurrency

In [4]:
import trade
from utils.concurrency import *
from utils.time_ops import  *
from utils.pandas_ops import *
from utils import *

Configurations

In [43]:
from conf.setup_log import bLog,log, ic

In [1]:
from conf.binance_client import client,async_client

Test configs

In [15]:
ic("Begging to begin")

'03/25/2023 16:24:17' - INFO - 16:24:17'Begging to begin'


'Begging to begin'

In [57]:
from trade.account import *


balance()

ImportError: cannot import name 'to_date' from 'utils.time_ops' (/Users/hiro/pythoneerHiro/binance/utils/time_ops.py)

In [3]:
from binance import AsyncClient, BinanceSocketManager
import asyncio

async def handle_socket_message(msg):
    print(msg)

async def start_websocket(symbol, interval):
    client = async_client
    bm = BinanceSocketManager(client)
    kline_socket = bm.kline_socket(symbol, interval)

    async with kline_socket as stream:
        while True:
            try:
                msg = await asyncio.wait_for(stream.recv(), timeout=1.0)
                await handle_socket_message(msg)
            except asyncio.TimeoutError:
                continue

await start_websocket('BTCUSDT', '1m')

{'e': 'kline', 'E': 1679919295842, 's': 'BTCUSDT', 'k': {'t': 1679919240000, 'T': 1679919299999, 's': 'BTCUSDT', 'i': '1m', 'f': 5394808, 'L': 5394832, 'o': '27906.94000000', 'c': '27905.91000000', 'h': '27906.98000000', 'l': '27905.90000000', 'v': '0.48986100', 'n': 25, 'x': False, 'q': '13670.34040025', 'V': '0.24613000', 'Q': '6868.73656649', 'B': '0'}}


CancelledError: 

In [44]:
import pandas as pd
import numpy as np
import talib
import time
import asyncio
import unsync
from queue import Queue
from dask import delayed,compute
from unsync import unsync
import math

In [59]:
from trade.coin import *

ImportError: cannot import name 'to_date' from 'utils.time_ops' (/Users/hiro/pythoneerHiro/binance/utils/time_ops.py)

In [ ]:
from exchange_data import SocketHandler
from events import SignalHandler
from strategies import Strategy
from exchange_data import Socket


class Trader(Strategy,SignalHandler,SocketHandler,Socket):

  args = {}
  # for i in ['OS','OL','CS','CL'] : args[i] = {}

  def __init__(self,crypto,profit=1,hedge=1,amount=''):
    l_h = 0 if hedge in [0,2] else 1
    s_h = 0 if hedge in [0,3] else 1
    self.CRYPTO = crypto
    self.BALANCE = balance('USDT')
    self.AMOUNT = amount if amount else min_q(crypto)
    self.PROFIT = profit #self.BALANCE*profit*.01  # percentage
    self.LONG = Position('L',crypto,self.PROFIT,l_h)
    self.SHORT =Position('S',crypto,self.PROFIT,s_h)
    self.PRECISION = precision(crypto)
    self.SYMBOL = symboler(self.CRYPTO)


  def update_args(self,pos,**args):
    self.args[pos].update(args)
    mode,ps = pos
    if mode=='O':
      if ps=='S': self.SHORT.open_args = args
      else: self.LONG.open_args = args
    else:
      if ps=='S': self.SHORT.close_args = args
      else: self.LONG.close_args = args

  def set_args(self,pos,**args):
    self.args[pos]={}
    self.update_args(pos,**args)

  async def trade(self,period=math.inf):
    self.buys = Queue(100)
    self.sells = Queue(100)
    r_trades = recent_trades(self.CRYPTO)
    self.CLOSES = r_trades['price'].to_numpy()

    self.macding() #My cur strategy
    self.buying()
    self.selling() #next is the opening
    max = await self.market_socket()
    handler = self.handle_socket
    await self.socket_handler(max,handler,period)

In [ ]:

#  dir(client)

In [ ]:
def cancel_order(coin,id=''):
  d = {'orderId':id,'symbol':symboler(coin)}
  return client.futures_cancel_all_open_orders(**d) #{'code': 200, 'msg': 'The operation of cancel all open order is done.'}

def order(coin,side,mode,quantity='',price='',profit='',loss='',trail='',PRECISION=''):
  p,q,bP,qP,minP,maxP,minQ = PRECISION or precision(coin)

  if quantity: quantity = round(max(quantity,minQ),q)

  d = {'type':'MARKET','symbol':symboler(coin), "quantity":quantity,'positionSide':mode}

  if price:
    price = round(price,p)
    price = min(maxP,price)
    price = max(minP,price)
    d['type'] = 'LIMIT'
    d['price'] = price
    d['timeInForce']='GTC'

  if profit:
        d['type']='TAKE_PROFIT' if price else 'TAKE_PROFIT_MARKET'
        d['stopPrice'] = round(profit,p)
  elif loss:
    d['type']='STOP' if price else 'STOP_MARKET'
    d['stopPrice'] = round(loss,p)
  elif trail:
        d['type'] = 'TRAILING_STOP_MARKET'
        d['callbackRate'] = trail
        d['activationPrice'] = price

  if side=='OPEN' :
    d['side']  = 'BUY' if mode == 'LONG' else 'SELL'
  else :
     d['side']  = 'SELL' if mode == 'LONG' else 'BUY'
     if quantity=='' and (profit or loss):
        d['closePosition'] = True
        del d['quantity']

  log.info(d)

  x=client.futures_create_order(**d)
  df = pd.DataFrame(x,index=[0]).rename(columns={"":"position"})
  drop_cols(df,*['status','clientOrderId','avgPrice','executedQty','cumQty','cumQuote','timeInForce','reduceOnly','workingType','priceProtect','origType','updateTime'])
  ncol = ['activatePrice','origQty','price','priceRate','stopPrice']
  for i in ncol:
    if i in df.columns:df[i] = num_col(df[i])
  x = df.iloc[0].to_dict()
  return x
#https://dev.binance.vision/t/what-does-the-percent-price-filter-mean/134

In [53]:
!date

Sat Mar 25 20:17:04 IST 2023


In [ ]:
@unsync
async def main():
  ic.disable()
  opening()
  closing()
  period = math.inf      
  BALANCE = balance('USDT')
  # AMOUNT = .01         #hedge -0 (open disabled) 1- long&short 2-long only 3-short only
  T1 = Trader('LTC',2,hedge=1)
  T2 = Trader('ETH',.02,hedge=1)
  T3 = Trader('BTC',2,hedge=1)

  T4 = Trader('EOS',.3,hedge=1)
  T5 = Trader('GALA',.2,hedge=1)
  T6 = Trader('HOT',.2,hedge=0)
  T7 = Trader('ANKR',.3,hedge=0)
  
  T8 = Trader('WAVES',.03,hedge=1)
  T9 = Trader('REEF',.01,hedge=0)
  T10 = Trader('MANA',.08,hedge=1)
  T11 = Trader('DENT',.03,hedge=1)
  T12 = Trader('AKRO',.08,hedge=1)
  d = {
      'LTC':(.02,1),
      'ETH':(.02,1),
      'BTC':(.1,1)
  }
  args = {
      'OS':{
          'price':'',
          'trail':.1
      },
      'OL':{
          'trail':.1
      },
     'CS':{
          'price':'',
         'trail':.1
      },
      'CL':{
         'price':'',
         'trail':.2
      }
    }
  # trades = [T8,T9,T10,T11,T12]
  trades = [T9,T11]
  for pos,kwargs in args.items():
    for trade in trades:
      trade.set_args(pos,**kwargs)
      leverage(trade.coin) #setting max lev

  await  asyncio.gather(*[i.trade() for i in trades])

In [ ]:
client = gen_client(True)


In [ ]:
socket = socket_manager().result()

main().result()

BinanceAPIException: ignored

Archived Interests

In [ ]:
##THIS is overall i guess its bitcoins

import requests

url = "https://fear-and-greed-index.p.rapidapi.com/v1/fgi"

headers = {
    'x-rapidapi-host': "fear-and-greed-index.p.rapidapi.com",
    'x-rapidapi-key': "56c20be206msh6f859fcbad9eef4p1810efjsn69b5010a55d2"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

In [ ]:
#I need to find fgi using macd at larger intervals & take the avg

#get k-lines - dont just take closes, take opens,and avg . %of that two

#overall macd should be used to say whether the market is bulling or bearish !!!....

#async fc or keepAlive should do the job

#also take coin fgi into consideration

In [ ]:
df= income(1,1)
print(df)
df.groupby(by=['symbol']).sum()

In [ ]:
import subprocess

proc = subprocess.Popen(
            ic(),
            shell=False,
            stdout=subprocess.PIPE,
        )

while proc.poll() is None:
   ic(proc.stdout.readline() + '<br/>\n')

https://binance-docs.github.io/apidocs/futures/en/#20xx-processing-issues

In [ ]:
import sys

with open(sys.stdout,'r') as f1:
     c = f1.read()
     ic(c)

Work Area

In [ ]:
!date

In [ ]:
!date

In [ ]:
income()

In [ ]:
income(False,False)

In [ ]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean() #properties of df
    exp2 = price.ewm(span = slow, adjust = False).mean()
    df = pd.DataFrame()
    df['price'] = price
    df['macd'] = exp1 - exp2
    df['signal'] = df['macd'].ewm(span = smooth, adjust = False).mean()
    df['hist'] = df['macd'] - df['signal']
    return df

cd = pd.DataFrame(CLOSES)
mac1 = get_macd(cd, 26, 12, 9)

In [ ]:
import multiprocessing #standard in python 3
from threading import enumerate

print(multiprocessing.cpu_count())
ic(threading.active_count())

for thread in enumerate(): 
    print(thread.name)



In [ ]:
p = lambda x : print("SDSD")
def timeIT(fc,*args):
  k=[]
  for i in range(1000):
    x = time.time()
    y = time.time()
    fc(*args)
    k.append(x-y)
  z = np.array(k).mean()
  ic(fc.__name__,'took',z)
  return z

# timeIT(p)
cd = pd.DataFrame(CLOSES)
timeIT(get_macd,cd,26,12,9)
timeIT(talib.MACD,CLOSES,12,26,9) #Talib is faster apparently...

In [ ]:
def async_run(fc):
  def inner(*args,**kwargs):
    coroutine = fc(*args,**kwargs)
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  
        loop = None
        log.debug('Starting new event loop')
        asyncio.run(coroutine)
    else:
        log.debug('Async event loop already running. Adding coroutine to the event loop.')
        tsk = loop.create_task(coroutine)
        tsk.add_done_callback(lambda t: print(f'Task done with result={t.result()}'))
  return inner

In [ ]:
    # @unsync(cpu_bound=True)


In [ ]:

@async_run
async def c(v):
  while 1:
    ic(v,122)
    await asyncio.sleep(3)

async def d(v):
  print(v)


In [ ]:
c(1)

In [ ]:
loop = asyncio.get_running_loop()

In [ ]:
c = CLOSES 
m,s,histogram = talib.MACD(c, fastperiod=12, slowperiod=26, signalperiod=9)
frames = talib.MACD(c, fastperiod=12, slowperiod=26, signalperiod=9)
# df = pd.concat([m,s,histogram], join = 'inner', axis = 1)
df = pd.DataFrame(frames).transpose()
df.columns = ['macd','signal','histogram']
# df.dropna(inplace=True)
df


# m,s = df.iloc[0,:2]

# net = m-s
# if m>]

# prev_net = net
# net = m-s
 

Ploting macd plus our Strategy

In [ ]:
def implement_macd_strategy(prices, data):    #for ploting
    buy_price = []
    sell_price = []
    macd_signal = []
    signal = 0

    for i in range(len(data)):
        if data['macd'][i] > data['signal'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif data['macd'][i] < data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)
            
    return buy_price, sell_price, macd_signal
            
buy_price, sell_price, macd_signal = MACDStrategy(df['price'], df)


In [ ]:
import matplotlib.pyplot as plt 


plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('fivethirtyeight')

def plot_macd(prices, macd, signal, hist):
    ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

    ax1.plot(prices)
    ax2.plot(macd, color = 'grey', linewidth = 1.5, label = 'MACD')
    ax2.plot(signal, color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

    for i in range(len(prices)):
        if hist[i]<0:
            ax2.bar(prices.index[i], hist[i], color = '#ef5350')
        else:
            ax2.bar(prices.index[i], hist[i], color = '#26a69a')

    plt.legend(loc = 'lower right')
df = mac1
l = [df[i].to_list() for i in df.columns][1:]
plot_macd(df['price'],*l)


In [ ]:
ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

ax1.plot(df['price'], color = 'skyblue', linewidth = 2, label = 'price')
ax1.plot(df.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
ax1.plot(df.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
ax1.legend()
ax1.set_title('df MACD SIGNALS')
ax2.plot(df['macd'], color = 'grey', linewidth = 1.5, label = 'MACD')
ax2.plot(df['signal'], color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

for i in range(len(df)):
    if str(df['hist'][i])[0] == '-':
        ax2.bar(df.index[i], df['hist'][i], color = '#ef5350')
    else:
        ax2.bar(df.index[i], df['hist'][i], color = '#26a69a')
        
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
df['price'].tail()

In [ ]:
df

In [ ]:
positions()

In [ ]:
while 1:
  print(positions()['unRealizedProfit'].sum())
  time.sleep(5)


In [ ]:
# try:
#   loop = asyncio.get_running_loop()
#   main().result()
# except Exception as e:
#   lp = asyncio.get_event_loop()
#   # lp.stop()
#   print('Crash')
#   # loop.stop()`
#   print(e)

https://www.shanelynn.ie/using-python-threading-for-multiple-results-queue/
https://stackoverflow.com/questions/20056300/python-print-output-of-each-thread-to-seperate-file-no-processes

In [ ]:
#https://medium.com/codex/algorithmic-trading-with-macd-in-python-1c2769a6ad1b

In [ ]:
# bearish when macd <0
#bullish macd crosses >0


In [ ]:
#bearosj macd croses  bellow signal line

In [ ]:
# try:
#     loop = asyncio.get_running_loop()
# except RuntimeError:  # 'RuntimeError: There is no current event loop...'
#     loop = None

# if loop and loop.is_running():
#     print('Async event loop already running. Adding coroutine to the event loop.')
#     tsk = loop.create_task(main())
#     # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
#     # Optionally, a callback function can be executed when the coroutine completes
#     tsk.add_done_callback(
#         lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
# else:
#     print('Starting new event loop')
#     asyncio.run(main())

In [ ]:
#  miniticker_socket(self, update_time: int = 1000):
  # aggtrade_socket(
  #     symbol_ticker_socket(self, symbol: str):
  #     ticker_socket(self):
  #         def index_price_socket(self, symbol: str, fast: bool = True):


Prediction


In [ ]:
from fbprophet import Prophet


def predict(df,freq='1',period=60,mode='L',his=True):
  d = {'L':'linear','log':'logistic'}
  m = Prophet(growth=d.get(mode),daily_seasonality=True)

  m.fit(df)
  fut = m.make_future_dataframe(periods=period,freq=f'{freq}min',include_history=his)
  fore = m.predict(fut)
  fore['ds']  = fore['ds'].values.tolist()
  return fore

Historical 

In [ ]:
import numpy as np

df=history('BTC','1m','1 day ago')
# df['date']  = pd.to_datetime(df['date']).dt.time
make_history(df)

# make_history(df)

# df['ds'] = df['ds'].apply(lambda x: pd.Timestamp(x))
# df['ts'] = df.ds.values.astype(np.int64) 

In [ ]:
df['ds'] = df.ds.values.tolist()

In [ ]:
df.loc[df['ds']==1642402980000000000]

In [ ]:
ff = predict(df,1,100,his=False)
# ff['ds']=pd.to_datetime(df['ds']).dt.time
ff
tr


In [ ]:
!date

In [ ]:
df[1440:]

In [ ]:
xf=history('EOS','1m','1 day ago')
make_history(xf)
predict(xf)

In [ ]:
x = df['ds'].iloc[1] -today()
x

In [ ]:
#Streaming

def listenKey(key=None,op=0):
  if op: return client.futures_stream_close(key)
  elif key==None:return client.futures_stream_get_listen_key()
  return client.futures_stream_keepalive(key)


In [ ]:
predict(df)

In [ ]:
predict(xf)

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(df)
fut = m.make_future_dataframe(periods=20,freq=f'{1}min',include_history=True)
fore = m.predict(fut)
fore.tail()

In [ ]:

m.plot(fore); #close->low 

In [ ]:
m.plot(fore); 

In [ ]:
m.plot(fore,figsize=(20,8));

In [ ]:
m.plot(fore,figsize=(20,8)); #linear

In [ ]:
m.plot(fore);

In [ ]:
m.plot_components(fore);

In [ ]:
fore[len(df):][['yhat','yhat_lower','yhat_upper','ds']]

In [54]:
%magic

Bibiliography

https://github.com/sammchardy/python-binance/blob/master/binance/client.py

https://colab.research.dfe.com/drive/1knvw5uUReCjrN7dbZNP-OzYs1OY5Mmc4#scrollTo=y_RiMszHFCZ_

https://algotrading101.com/learn/binance-python-api-guide/

https://medium.com/analytics-vidhya/futures-trading-with-python-binance-d738c71e17b5

https://binance-docs.github.io/apidocs/futures/en/#new-future-account-transfer

https://readthedocs.org/projects/python-binance/downloads/pdf/latest/


https://testnet.binancefuture.com/en/futures/BTCUSDT


